In [ ]:
import os
import json
#Tokenize text
from konlpy.tag import Kkma 
from konlpy.utils import pprint 
import pandas as pd

In [ ]:


class parseText:
    def __init__(self):
        self.dir=""
        self.stopwords=pd.DataFrame(colums=['vocab','pos'])
        self.data=[]
        self.parsedData=[]
        self.kkma=None
        
    def openAllJson(self,dirname): #"./2020-02-019.도서자료요약_Sample"
        self.dir+=dirname## get path
        #1) open json file
        jsonobj=[]
        for file in os.listdir(dirname):
            jsonfd=open(dirname+"/"+file, encoding="utf-8")## get file descriptor
            jsonstr=jsonfd.read()## get str
            jsonobj.append(jsonstr)
        
        return jsonobj
    
    def organizeFile(self,jsonobj):
		#2) select passages and summaries
        return {'id':jsonobj["passage_id"],'passage':jsonobj["passage"],'summary':jsonobj["summary"]}
    
    def organaizeFiles(self,dirname):
        jsonobj=self.openAllJson(dirname)
        for i in range(len(jsonobj)):
            self.data.append(self.organizeFile(jsonobj[i]))
    
    def setStopwords(self,path):#stopwords.txt
        self.kkma=Kkma()
        f=open(path,encoding="utf-8")
        self.stopwords=pd.concat((self.stopwords,pd.DataFrame(self.kkma.pos(f.read()),columns=['vocab','pos'])),axis=0)
    
    def preprocessFile(self,data):
        passage=data['passage']
        word_dict=pd.DataFrame(columns=['vocab','pos','mundan_id','sent_id','eojeol_id'])
        for i,mundan in enumerate(passage.split('\n')): #i는 n 번째 문단이라는 뜻
            position=(i)
            for j, sent in enumerate(mundan.split('.')):#j 는 n번째 문장이라는 뜻
                position+=(j)
                for k, word in enumerate(sent.split(' ')):#k는 n번째 어절이라는 뜻 
                    position+=(k)
                    passage_hs=kkma.pos(word) #list in list
                    for hs in passage_hs:
                        current=pd.DataFrame(position+hs,columns=['vocab','pos','mundan_id','sent_id','eojeol_id'])
                        word_dict=pd.concat((word_dict,current),axis=0)
        return word_dict


    def preprocessFiles(self,stoppath):
        self.setStopwords(stoppath)
        #new passage object with words-->words & PoS tuples / only NNG, VV
        for d in self.data:
            word_PoS_pos=self.preprocessFile(d)
            #품사,stopwords 필터, 재정렬
            filtered=word_PoS_pos[(word_PoS_pos.pos=='NNG')|(word_PoS_pos.pos=='VV')&(~word_PoS_pos['vocab'].isin(list(self.stopwords.vocab)))].reset_index()
            self.parsedData.append(filtered)
        return self.parsedData
    
    def doAll(self):
        dirname=input('dirname of jsonobj directory')
        stoppath=input('dirname of stopwords txt')
        self.organaizeFiles(dirname)
        self.preprocessFiles(stoppath)




In [ ]:
from konlpy.tag import Kkma
kkma=Kkma()
f=open('testParse.txt','r')
fstr=f.read()
fstrs=fstr.split('\n')
f_wpos=kkma.pos(fstr) #전체 문단에 대한 pos
fstrs_wpos=[kkma.pos(s) for s in fstrs] #각 문장에 대한 pos

for sent in fstrs_wpos:
	a=pd.DataFrame(sent,columns=['vocab','pos'])
	a['sent_id']=1